In [1]:
   pip install simpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import simpy
import random
import statistics

In [3]:
class CentroUrgencias:
    def __init__(self, env, num_medicos):
        # Guardar el entorno
        self.env = env
        
        # Crear el recurso médico: capacidad = número de médicos disponibles
        self.medico = simpy.Resource(env, capacity=num_medicos)
        
        # Lista donde se almacenará el tiempo de espera de cada paciente
        self.tiempos_espera = []

    def atencion_paciente(self):
        # Registrar la hora en la que el paciente llega al sistema
        llegada = self.env.now

        # Solicitar turno para ser atendido
        with self.medico.request() as turno:
            yield turno  # El paciente espera si no hay médico disponible

            # Calcular el tiempo de espera
            espera = self.env.now - llegada
            self.tiempos_espera.append(espera)

            # Tiempo de atención con distribución normal: media 20 min, sd 5 min
            duracion = random.normalvariate(20, 5)
            yield self.env.timeout(duracion)

    def llegada_pacientes(self):
        # Generar pacientes durante toda la simulación
        while True:
            # Crear un proceso por cada paciente
            self.env.process(self.atencion_paciente())

            # Tiempo entre llegadas: exponencial con media = 15 minutos
            tiempo_llegada = random.expovariate(1 / 15)
            yield self.env.timeout(tiempo_llegada)

In [4]:
#Ejecutar la Simulacion

In [5]:
def ejecutar_simulacion(num_medicos):
    env = simpy.Environment()

    hospital = CentroUrgencias(env, num_medicos)

    # Iniciar proceso de llegada de pacientes
    env.process(hospital.llegada_pacientes())

    # Ejecutar simulación durante 8 horas = 480 minutos
    env.run(until=480)

    # Calcular tiempo promedio de espera
    if len(hospital.tiempos_espera) > 0:
        return statistics.mean(hospital.tiempos_espera)
    else:
        return 0

In [6]:
espera_1 = ejecutar_simulacion(1)
print(f"Tiempo promedio de espera con 1 médico: {espera_1:.2f} minutos")

Tiempo promedio de espera con 1 médico: 38.17 minutos


In [7]:
espera_2 = ejecutar_simulacion(2)
print(f"Tiempo promedio de espera con 2 médicos: {espera_2:.2f} minutos")

Tiempo promedio de espera con 2 médicos: 4.78 minutos


In [8]:
if espera_2 < espera_1:
    print("Conclusión: Se recomienda usar 2 médicos para reducir el tiempo de espera.")
else:
    print("Conclusión: Con 1 médico es suficiente.")

Conclusión: Se recomienda usar 2 médicos para reducir el tiempo de espera.
